In [ ]:
!pip install torch transformers
!pip install -U datasets
!git clone https://github.com/davidjurgens/citation-function

In [2]:
import os
with open('api_openrouter.txt', 'r') as f:
  API=f.read()


In [3]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

/home/mcherif/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
for text,intent in zip(test['text'],test['intent']):
    print('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
            + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
    print(text+'\n\n')

You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:From the following list: 0:BACKGROUND 
 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE

Resnik ( 1995 ) reported a correlation of r = .9026.10 The results are not directly comparable , because he only used noun-noun pairs , words instead of concepts , a much smaller dataset , and measured semantic similarity instead of semantic relatedness .


You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the int

In [5]:
type(int(test['intent'][0]))

int

In [6]:
dico_rez={
    0:[],
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
}

In [7]:
import requests
import json
import os

les_rez=[]
dico_rez={0:[],1:[],2:[],3:[],4:[],5:[]}

url = "https://openrouter.ai/api/v1/chat/completions"


headers = {
  "Authorization": f"Bearer {API}",
  "Content-Type": "application/json"
}




for text,intent in zip(test['text'],test['intent']):
  PROMPT=('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
          + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
  print(text+'\n\n')
  try:
    payload = {
        "model": "deepseek/deepseek-r1-0528",
      "messages": [
        {
          "role": "user",
          "content": PROMPT
        }
      ]
        ,
    #   "provider": {
    #     "allow_fallbacks": True,
    #     "quantizations": [
    #       "fp8"
    #     ]
    # },
        "reasoning": {

      "enabled": True  # Use high reasoning effort

  }
    }
    response = requests.post(url, headers=headers, json=payload)
    les_rez.append(response.json())
    dico_rez[int(intent)].append(response.json())
  except:
    print("error")
    continue
  print(response.json())

Resnik ( 1995 ) reported a correlation of r = .9026.10 The results are not directly comparable , because he only used noun-noun pairs , words instead of concepts , a much smaller dataset , and measured semantic similarity instead of semantic relatedness .


{'id': 'gen-1758722852-fifu5enUkWICdw6p8CAs', 'provider': 'Nebius', 'model': 'deepseek/deepseek-r1-0528', 'object': 'chat.completion', 'created': 1758722852, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '3', 'refusal': None, 'reasoning': 'Alright, let\'s tackle this problem. So, I have this citation text: "Inspired by the success of Smith et al. [12]..." That\'s the key piece here. \n\nFirst, I need to recall the categories: \n0 for Background, \n1 for Uses, \n2 for Compares or Contrasts, \n3 for Motivation, \n4 for Continuation, \n5 for Future.\n\nLooking at the word "inspired" – that\'s a clue. When a paper says it was inspired by pr

In [12]:
for i in range(0,6):
    for elt in dico_rez[i]:
        with open(f'DATASET5/results_openrouter_intent_{i}.txt', 'a',encoding='utf-8') as f:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')

In [13]:
with open("dico.json", "w", encoding="utf-8") as f:
    json.dump(dico_rez, f, ensure_ascii=False, indent=4)